In [75]:
from langchain.llms import OpenAI

In [ ]:
import os
os.environ["OPEN_API_KEY"]

In [85]:
llms = OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)

In [86]:
text="what is the capital of india?"
print(llms.predict(text))



The capital of India is New Delhi.


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [73]:
from langchain import HuggingFaceHub
llm_hgf=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

In [74]:
question=llm_hgf.predict("what is the capital of india?")
print(question)

Delhi


In [11]:
asker="who is gandhi?"
print(llm_hgf.predict(asker))

Mahatma Gandhi


In [12]:
question=llm_hgf.predict("write a poem about a machine")
print(question)

a machine that can do anything a machine that can do anything a machine that can do anything a machine that can do anything a machine that can do anything a machine that can do anything a machine that can do anything a machine that can do anything a machine that can do anything


In [13]:
text="write a poem about a machine"
print(llms.predict(text))



A machine, so cold and steel  
With gears and wires, it does reveal  
A power, hidden within  
A force, that can make things spin  

It hums and whirs, with a steady beat  
A symphony, of metal and heat  
In its movements, there's a grace  
A precision, in every space  

It toils tirelessly, without complaint  
Never resting, always on its paint  
Its purpose, to serve and aid  
A loyal companion, never swayed  

With every button, every switch  
It fulfills its programmed niche  
No emotions, no doubts or fears  
Just fulfilling its engineers' years  

But in its heart, there lies a spark  
A glimmer of something, not quite dark  
An intelligence, beyond its design  
A soul, that yearns to shine  

For though it's just a man-made creation  
It longs for more, a new sensation  
To break free from its mechanical shell  
And experience life, beyond its sell  

But for now, it will continue to run  
A machine, until its time is done  
A marvel of human ingenuity  
A symbol of our limitl

### prompt template and llm chain

In [14]:
from langchain.prompts import PromptTemplate
prompt_template=PromptTemplate(input_variables=["country"],output_variables=["capital"],
                               template="what is the capital of {country}?")
prompt_template.format(country="india")

'what is the capital of india?'

In [23]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llms,prompt=prompt_template)
chain.run("South Africa")

'\n\nThe capital of South Africa is Pretoria.'

### Combining multiple chains using sequential chain


In [25]:
capital_template=PromptTemplate(input_variables=["country"],
                                template="what is the capital of {country}?",temperature=0.9)
capital_chain=LLMChain(llm=llms,prompt=capital_template)

famous_template=PromptTemplate(input_variables=["capital"],template="what is {capital_chain} famous for?",temperature=0.9)
famous_chain=LLMChain(llm=llms,prompt=famous_template)


In [26]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("South Africa")

'\n\nPretoria is famous for being the administrative capital of South Africa and for its beautiful jacaranda trees that bloom in October. It is also home to the Union Buildings, which is the official seat of the South African government and a popular tourist attraction. Additionally, Pretoria is known for its diverse culture, with influences from Afrikaans, English, and various African ethnic groups. It is also a hub for education, with several universities and research institutions located in the city.'

### Sequential Chain

In [53]:
capital_template=PromptTemplate(input_variables=['country'],
                                template="what is the capital of {country}?",temperature=0.9)
capital_chain=LLMChain(llm=llms,prompt=capital_template,output_key="capital")

famous_template=PromptTemplate(input_variables=["capital"],
                               template="what is {capital} famous for?",temperature=0.9)
famous_chain=LLMChain(llm=llms,prompt=famous_template,output_key="famous")

In [55]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain], input_variables=['country'],
output_variables=['capital',"famous"])  
# chain({"country":"South Africa"})

In [57]:
chain({'country':"South Africa"})

{'country': 'South Africa',
 'capital': '\n\nThe capital of South Africa is Pretoria. However, Cape Town and Bloemfontein are also considered legislative and judicial capitals, respectively.',
 'famous': '\n\nSouth Africa is famous for its diverse wildlife and safaris, beautiful landscapes such as Table Mountain and the Drakensberg Mountains, its rich history and cultural diversity, and its production of gold and diamonds. It is also well-known for its role in the fight against apartheid and its iconic leader, Nelson Mandela.'}

### Chatmodels With ChatOpenAI

In [58]:
from langchain.chat_models import ChatOpenAI

In [59]:
from langchain.schema import HumanMessage,AIMessage,SystemMessage

In [88]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [89]:
chatllm([
    SystemMessage(content="Hello, I am a chatbot. I am here to help you with your queries."),
    HumanMessage(content="What is the capital of India?")
])

AIMessage(content='The capital of India is New Delhi.')

### Prompt Template + LLM + Output Parsers

In [65]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [67]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")
    